# Class project

### 3. Anime face generation:
Given the image of anime faces as follows.
https://www.kaggle.com/splcher/animefacedataset
Use Variational Autoencoder and GAN to generate faces of anime characters.
Compare the performance of these two models.

> This is generative model using Generative Adversarial Network that consists of Generator and discriminator networks.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
!pip install -q kaggle

os.environ['KAGGLE_USERNAME'] = 'nuttrawitra' 
os.environ['KAGGLE_KEY'] = '' 
! kaggle datasets download -d splcher/animefacedataset

! unzip animefacedataset.zip

In [1]:
import os, sys
import shutil
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import  Dense, Dropout, LeakyReLU,  BatchNormalization, Conv2DTranspose, Conv2D, Reshape, Flatten
from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:
os.mkdir('anime')

> Using ImageDataGenerator, the Keras image preprocessing, to get anime data set directly from folder 'image'. 

In [2]:
# shutil.move('images', 'anime')  

IDG = ImageDataGenerator(rescale=1./255)
train_data = IDG.flow_from_directory(
    'anime',
    target_size=(64, 64),
    batch_size=63565,
    shuffle = True,
    class_mode = 'input',
    subset='training') 

Found 63565 images belonging to 1 classes.


In [5]:
def Generator():
    generator=Sequential(name='Generator')
    
    generator.add(Dense(16384,input_dim=16, name='gen_input'))
    generator.add(LeakyReLU(0.2))
    generator.add(Reshape((16,16,64)))

    generator.add(Conv2D(64, 4, padding='same'))
    generator.add(LeakyReLU(0.2))

    generator.add(Conv2DTranspose(128, 4, strides=2, padding='same'))
    generator.add(LeakyReLU(0.2))
    generator.add(BatchNormalization(momentum=0.8))
    
    # generator.add(Conv2DTranspose(256, 4, strides=2, padding='same'))
    # generator.add(LeakyReLU(0.2))
    # generator.add(BatchNormalization(momentum=0.8))
    
    generator.add(Conv2DTranspose(128, 4, strides=2, padding='same'))
    generator.add(LeakyReLU(0.2))
    generator.add(BatchNormalization(momentum=0.8))
    
    generator.add(Dense(3, activation=tf.keras.activations.tanh , name='gen_output'))

    generator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
    return generator
generator=Generator()
generator.summary()

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gen_input (Dense)            (None, 16384)             278528    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 16384)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        65600     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 128)       131200    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32, 32, 128)       0 

In [6]:
def Discriminator():
    discriminator=Sequential(name='Discriminator')

    discriminator.add(Input(shape=(64,64,3)))

    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(128, 4, name='disc_input'))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.2))
       
    discriminator.add(Conv2D(128, 4, 2))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.2))
       
    # discriminator.add(Conv2D(256, 4, 2))
    # discriminator.add(LeakyReLU(0.2))
    # discriminator.add(Dropout(0.2))

    discriminator.add(Conv2D(64, 4, 2))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.2))
    
    discriminator.add(Flatten())
    discriminator.add(Dense(units=1, activation='sigmoid', name='disc_output'))
    
    discriminator.compile(loss='binary_crossentropy',  optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
    return discriminator
discriminator =Discriminator()
discriminator.summary()

Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
leaky_re_lu_12 (LeakyReLU)   (None, 64, 64, 3)         0         
_________________________________________________________________
disc_input (Conv2D)          (None, 61, 61, 128)       6272      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 61, 61, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 61, 61, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 128)       262272    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 29, 29, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 29, 128)     

In [7]:
def GAN(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(16,), name='GAN_input')
    gan= Model(gan_input, discriminator(generator(gan_input)) , name='GAN_model')
    gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
    return gan
gan =GAN(discriminator, generator)
gan.summary()

Model: "GAN_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GAN_input (InputLayer)       [(None, 16)]              0         
_________________________________________________________________
Generator (Sequential)       (None, 64, 64, 3)         739011    
_________________________________________________________________
Discriminator (Sequential)   (None, 1)                 410497    
Total params: 1,149,508
Trainable params: 738,499
Non-trainable params: 411,009
_________________________________________________________________


In [8]:
def plot_generated_images(epoch, generator, examples=16, dim=(4,4), figsize=(30,30)):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 16])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(16,64,64,3)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_new/gan_generated_image%d.png' %epoch)

In [ ]:
from tqdm import tqdm
data = next(train_data)
data = data[0]

def realimage():
    image_batch = []   
    for c in range(64):
        image_batch.append(data[np.random.randint(low=0,high=63565)])
    image_batch = np.array(image_batch)
    # image_batch = image_batch.reshape(64,64,64,3)
    return image_batch

def training(epochs, batch_size=64):
    
    #Loading the data
    batch_count = 63565 / batch_size
    
    # Creating GAN
    generator= Generator()
    discriminator= Discriminator()
    gan = GAN(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
            noise= np.random.normal(0,1, [batch_size, 16])
            
            generated_images = generator.predict(noise)
#             image_batch = realimage()
 
            image_batch = data[np.random.randint(low=0,high=63565,size=batch_size)]
            
            X= np.concatenate((image_batch, generated_images))
            
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9

            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            noise= np.random.normal(0,1, [batch_size, 16])
            y_gen = np.ones(batch_size)
          
            discriminator.trainable=False
            
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)
training(4000,64)

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1


100%|██████████| 64/64 [00:30<00:00,  2.12it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] 

Epoch 2


  0%|          | 0/64 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1362


  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1363


 17%|█▋        | 11/64 [00:04<00:23,  2.27it/s]